In [35]:
from datetime import date
import pandas as pd
import numpy as np
import yaml
from sqlalchemy import create_engine

# Database connections 

In [36]:
# Cargar configuración
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

In [37]:
# Crear conexiones
url_fuente = f"postgresql://{config_fuente['user']}:{config_fuente['password']}@{config_fuente['host']}:{config_fuente['port']}/{config_fuente['dbname']}"
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"

# Crear los SQLAlchemy Engine
fuente_conn = create_engine(url_fuente)
bodega_conn = create_engine(url_bodega)

# SQL Query

In [38]:
query = """
SELECT 
    ha.servicio_id,
    ha.key_dim_fecha,
    ha.key_dim_cliente,
    ha.key_dim_mensajero,
    ds.key_dim_sede,
    df.dia_semana
FROM hecho_entrega_acumulado ha
JOIN dim_cliente dc ON ha.key_dim_cliente = dc.key_dim_cliente
JOIN dim_sede ds ON dc.cliente_id = ds.sede_id
JOIN dim_fecha df ON ha.key_dim_fecha = df.key_dim_fecha
WHERE ha.fecha_iniciado IS NOT NULL
"""

# Extract

In [45]:
# Leer datos y dimensiones
df_servicios = pd.read_sql(query, bodega_conn)

In [46]:
print("Total servicios extraídos:", len(df_servicios))

Total servicios extraídos: 28428


In [47]:
hecho_dia = df_servicios.groupby([
    'key_dim_fecha',
    'key_dim_cliente',
    'key_dim_sede',
    'key_dim_mensajero',
    'servicio_id'
]).size().reset_index(name='cantidad_servicios_dia')

In [48]:
print("\nEstadísticas de agregación:")
print(f"Total registros agregados: {len(hecho_dia)}")
print("\nDistribución de cantidad de servicios por día:")
print(hecho_dia['cantidad_servicios_dia'].value_counts().sort_index())


Estadísticas de agregación:
Total registros agregados: 27701

Distribución de cantidad de servicios por día:
cantidad_servicios_dia
1    27701
Name: count, dtype: int64


In [49]:
# Agregar fecha de carga
hecho_dia['saved'] = date.today()

# Verifications

In [50]:
# Verificar datos antes de cargar
print("\nMuestra de datos a cargar:")
print(hecho_dia.head())
print("\nInformación de columnas:")
print(hecho_dia.info())


Muestra de datos a cargar:
   key_dim_fecha  key_dim_cliente  key_dim_sede  key_dim_mensajero  \
0          261.0                7            16                0.0   
1          261.0                7            16               13.0   
2          261.0                7            16               13.0   
3          261.0                7            16               13.0   
4          261.0                7            16               13.0   

   servicio_id  cantidad_servicios_dia       saved  
0            7                       1  2024-11-10  
1            8                       1  2024-11-10  
2            9                       1  2024-11-10  
3           10                       1  2024-11-10  
4           11                       1  2024-11-10  

Información de columnas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27701 entries, 0 to 27700
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -

# Load

In [51]:
# Guardar en la bodega
hecho_dia.to_sql(
    'hecho_entrega_servicio_diaria', 
    bodega_conn, 
    if_exists='replace', 
    index_label='key_hecho_entrega_servicio_diaria'
)

701

In [53]:
# Verificación final
print("\nVerificación final:")
verificacion_query = """
SELECT 
    df.dia_semana,
    COUNT(*) as total_servicios
FROM hecho_entrega_servicio_diaria h
JOIN dim_fecha df ON h.key_dim_fecha = df.key_dim_fecha
GROUP BY df.dia_semana
ORDER BY df.dia_semana;
"""
print("\nServicios por día de la semana:")
print(pd.read_sql(verificacion_query, bodega_conn))


Verificación final:

Servicios por día de la semana:
   dia_semana  total_servicios
0           0             4203
1           1             5263
2           2             4814
3           3             5033
4           4             5153
5           5             2418
6           6              817


# SQL Querys to response requirements

In [ ]:
# Pregunta 1: ¿En qué meses del año los clientes solicitan más servicios?

'''
SELECT 
    EXTRACT(YEAR FROM df.fecha)::integer as año,
    CASE EXTRACT(MONTH FROM df.fecha)::integer
        WHEN 1 THEN 'Enero'
        WHEN 2 THEN 'Febrero'
        WHEN 3 THEN 'Marzo'
        WHEN 4 THEN 'Abril'
        WHEN 5 THEN 'Mayo'
        WHEN 6 THEN 'Junio'
        WHEN 7 THEN 'Julio'
        WHEN 8 THEN 'Agosto'
        WHEN 9 THEN 'Septiembre'
        WHEN 10 THEN 'Octubre'
        WHEN 11 THEN 'Noviembre'
        WHEN 12 THEN 'Diciembre'
    END as mes,
    COUNT(h.servicio_id) as total_servicios,
    ROUND((COUNT(h.servicio_id)::numeric / SUM(COUNT(h.servicio_id)) OVER (PARTITION BY EXTRACT(YEAR FROM df.fecha))) * 100, 2) || '%' as porcentaje_anual
FROM hecho_entrega_servicio_diaria h
JOIN dim_fecha df ON h.key_dim_fecha = df.key_dim_fecha
GROUP BY 
    EXTRACT(YEAR FROM df.fecha),
    EXTRACT(MONTH FROM df.fecha)
ORDER BY 
    año, 
    EXTRACT(MONTH FROM df.fecha);
'''

"\nSELECT \n    EXTRACT(YEAR FROM df.fecha)::integer as año,\n    CASE EXTRACT(MONTH FROM df.fecha)::integer\n        WHEN 1 THEN 'Enero'\n        WHEN 2 THEN 'Febrero'\n        WHEN 3 THEN 'Marzo'\n        WHEN 4 THEN 'Abril'\n        WHEN 5 THEN 'Mayo'\n        WHEN 6 THEN 'Junio'\n        WHEN 7 THEN 'Julio'\n        WHEN 8 THEN 'Agosto'\n        WHEN 9 THEN 'Septiembre'\n        WHEN 10 THEN 'Octubre'\n        WHEN 11 THEN 'Noviembre'\n        WHEN 12 THEN 'Diciembre'\n    END as mes,\n    COUNT(h.servicio_id) as total_servicios,\n    ROUND((COUNT(h.servicio_id)::numeric / SUM(COUNT(h.servicio_id)) OVER (PARTITION BY EXTRACT(YEAR FROM df.fecha))) * 100, 2) || '%' as porcentaje_anual\nFROM hecho_entrega_servicio_diaria h\nJOIN dim_fecha df ON h.key_dim_fecha = df.key_dim_fecha\nGROUP BY \n    EXTRACT(YEAR FROM df.fecha),\n    EXTRACT(MONTH FROM df.fecha)\nORDER BY \n    año, \n    EXTRACT(MONTH FROM df.fecha);\n"

In [ ]:
#Pregunta 2: ¿Cuáles son los días donde más solicitudes hay?

'''
SELECT 
    CASE df.dia_semana
        WHEN 0 THEN 'Lunes'
        WHEN 1 THEN 'Martes'
        WHEN 2 THEN 'Miércoles'
        WHEN 3 THEN 'Jueves'
        WHEN 4 THEN 'Viernes'
        WHEN 5 THEN 'Sábado'
        WHEN 6 THEN 'Domingo'
    END as dia,
    COUNT(h.servicio_id) as total_servicios,
    ROUND((COUNT(h.servicio_id)::numeric / SUM(COUNT(h.servicio_id)) OVER ()) * 100, 2) || '%' as porcentaje_total,
    ROUND(AVG(h.cantidad_servicios_dia), 2) as promedio_servicios_por_dia
FROM hecho_entrega_servicio_diaria h
JOIN dim_fecha df ON h.key_dim_fecha = df.key_dim_fecha
GROUP BY df.dia_semana
ORDER BY 
    CASE 
        WHEN df.dia_semana = 0 THEN 1
        WHEN df.dia_semana = 1 THEN 2
        WHEN df.dia_semana = 2 THEN 3
        WHEN df.dia_semana = 3 THEN 4
        WHEN df.dia_semana = 4 THEN 5
        WHEN df.dia_semana = 5 THEN 6
        WHEN df.dia_semana = 6 THEN 7
    END;
'''

"\nSELECT \n    CASE df.dia_semana\n        WHEN 0 THEN 'Lunes'\n        WHEN 1 THEN 'Martes'\n        WHEN 2 THEN 'Miércoles'\n        WHEN 3 THEN 'Jueves'\n        WHEN 4 THEN 'Viernes'\n        WHEN 5 THEN 'Sábado'\n        WHEN 6 THEN 'Domingo'\n    END as dia,\n    COUNT(h.servicio_id) as total_servicios,\n    ROUND((COUNT(h.servicio_id)::numeric / SUM(COUNT(h.servicio_id)) OVER ()) * 100, 2) || '%' as porcentaje_total,\n    ROUND(AVG(h.cantidad_servicios_dia), 2) as promedio_servicios_por_dia\nFROM hecho_entrega_servicio_diaria h\nJOIN dim_fecha df ON h.key_dim_fecha = df.key_dim_fecha\nGROUP BY df.dia_semana\nORDER BY \n    CASE \n        WHEN df.dia_semana = 0 THEN 1\n        WHEN df.dia_semana = 1 THEN 2\n        WHEN df.dia_semana = 2 THEN 3\n        WHEN df.dia_semana = 3 THEN 4\n        WHEN df.dia_semana = 4 THEN 5\n        WHEN df.dia_semana = 5 THEN 6\n        WHEN df.dia_semana = 6 THEN 7\n    END;\n"

In [115]:
#Pregunta 6: ¿Cuáles son las sedes que más servicios solicitan por cada cliente?
'''
SELECT 
    dc.nombre AS nombre_cliente,
    ds.nombre_sede,
    COUNT(h.servicio_id) AS total_servicios
FROM hecho_entrega_servicio_diaria h
JOIN dim_cliente dc ON h.key_dim_cliente = dc.key_dim_cliente
JOIN dim_sede ds ON h.key_dim_sede = ds.key_dim_sede
GROUP BY dc.nombre, ds.nombre_sede
ORDER BY total_servicios DESC;

'''



'\nSELECT \n    dc.nombre AS nombre_cliente,\n    ds.nombre_sede,\n    COUNT(h.servicio_id) AS total_servicios\nFROM hecho_entrega_servicio_diaria h\nJOIN dim_cliente dc ON h.key_dim_cliente = dc.key_dim_cliente\nJOIN dim_sede ds ON h.key_dim_sede = ds.key_dim_sede\nGROUP BY dc.nombre, ds.nombre_sede\nORDER BY total_servicios DESC;\n\n'